In [1]:
import os, logging, cv2
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf

In [50]:
image_size = (64, 64)  # Image size
num_classes = 2  # Number of classes
batch_size = 32  # Batch size

# Create TensorFlow training dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './homer_bart/',
    labels='inferred',
    label_mode='categorical',
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=42,
)

# Determine the total number of samples
total_samples = train_ds.cardinality().numpy()

# Calculate the number of samples for training and testing
train_size = int(0.9 * total_samples)
test_size = min(total_samples - train_size, 1)

# Split the dataset into training and testing
train_ds = train_ds.take(train_size)
test_ds = train_ds.take(test_size)

# Preprocess the data (normalize images)
preprocessed_data = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),  # Normalize images
])

# Apply the rescaling layer to both training and testing datasets
train_ds = train_ds.map(lambda x, y: (preprocessed_data(x), y))
test_ds = test_ds.map(lambda x, y: (preprocessed_data(x), y))

# Optionally, prefetch and cache datasets for better performance
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)


Found 269 files belonging to 2 classes.


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Input

In [25]:
model = Sequential()

model.add(Input(shape=(64, 64, 3)))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(8))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
model.fit(train_ds, epochs=60)

Epoch 1/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.5709 - loss: 1.0171
Epoch 2/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.6338 - loss: 0.6980
Epoch 3/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.6003 - loss: 0.6459
Epoch 4/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.6143 - loss: 0.6323
Epoch 5/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.6743 - loss: 0.6065
Epoch 6/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.7983 - loss: 0.5596
Epoch 7/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.8030 - loss: 0.5283
Epoch 8/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.7485 - loss: 0.5160
Epoch 9/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.7597 - loss: 0.4912
Epoch 10/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.6735 - loss: 0.5797
Epoch 11/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.7879 - loss: 0.4745
Epoch 12/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.7896 - loss: 0.4481
E

In [77]:
loss, accuracy = model.evaluate(test_ds)
print(f'Loss: {loss}, Accuracy: {accuracy}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 1.0000 - loss: 0.0026
Loss: 0.0025635804049670696, Accuracy: 1.0
